In [ ]:
!pip install scikit-learn tqdm transformers torch --quiet


In [ ]:
!pip install numpy==1.21.6 scipy==1.7.3 scikit-learn==1.1.3 scikit-multilearn==0.2.0 tqdm --quiet


ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement numpy==1.21.6 (from versions: 1.3.0, 1.4.1, 1.5.0, 1.5.1, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.7.1, 1.7.2, 1.8.0, 1.8.1, 1.8.2, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0.post2, 1.10.1, 1.10.2, 1.10.4, 1.11.0, 1.11.1, 1.11.2, 1.11.3, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 1.13.3, 1.14.0, 1.14.1, 1.14.2, 1.14.3, 1.14.4, 1.14.5, 1.14.6, 1.15.0, 1.15.1, 1.15.2, 1.15.3, 1.15.4, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.17.0, 1.17.1, 1.17.2, 1.17.3, 1.17.4, 1.17.5, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 1.18.5, 1.19.0, 1.19.1, 1.19.2, 1.19.3, 1.19.4, 1.19.5, 1.20.0, 1.20.1, 1.20.2, 1.20.3, 1.21.0, 1.21.1, 1.22.0, 1.22.1, 1.22.2, 1.22.3, 1.22.4, 1.23.0, 1.23.1, 1.23.2, 1.23.3, 1.23

In [ ]:
!pip install numpy --upgrade
!pip install scipy --upgrade

In [ ]:
import os
import numpy as np
import random
import time
import urllib.request
import bz2
from tqdm import tqdm

from scipy.sparse import vstack  # only for final train+val stacking if needed
from sklearn.metrics import f1_score, hamming_loss
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors


In [ ]:
# Download and read LexGLUE ECtHR (B) raw text dataset
base = "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multilabel/"
files_b = {
    "train": "ecthr_b_lexglue_raw_texts_train.txt.bz2",
    "val":   "ecthr_b_lexglue_raw_texts_val.txt.bz2",
    "test":  "ecthr_b_lexglue_raw_texts_test.txt.bz2"
}

os.makedirs("data", exist_ok=True)

def download_if_missing(fname):
    dest = os.path.join("data", fname)
    if not os.path.exists(dest):
        url = base + fname
        print("Downloading:", url)
        urllib.request.urlretrieve(url, dest)
    return dest

def read_raw_txt_bz2(path):
    texts, labels = [], []
    with bz2.open(path, "rt", encoding="utf-8", errors="replace") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if "\t" in line:
                labstr, txt = line.split("\t", 1)
            else:
                parts = line.split(maxsplit=1)
                if len(parts) < 2:
                    continue
                labstr, txt = parts
            labs = [l.strip() for l in labstr.split(",") if l.strip() != ""]
            labels.append(labs)
            texts.append(txt.replace("\n", " "))
    return texts, labels

# Download all files for ECtHR (B)
paths_b = {k: download_if_missing(v) for k, v in files_b.items()}

# Read them
train_texts_b, train_labels_b = read_raw_txt_bz2(paths_b["train"])
val_texts_b, val_labels_b     = read_raw_txt_bz2(paths_b["val"])
test_texts_b, test_labels_b   = read_raw_txt_bz2(paths_b["test"])

train_texts_b = train_texts_b[:1000]
train_labels_b = train_labels_b[:1000]

val_texts_b = val_texts_b[:200]
val_labels_b = val_labels_b[:200]

test_texts_b = test_texts_b[:200]
test_labels_b = test_labels_b[:200]

print(f"Train: {len(train_texts_b)}, Val: {len(val_texts_b)}, Test: {len(test_texts_b)}")


Train: 1000, Val: 200, Test: 200


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
import numpy as np
from sklearn.decomposition import PCA
# 1. Load LEGAL-BERT
tokenizer = AutoTokenizer.from_pretrained("nlpaueb/legal-bert-base-uncased")
model = AutoModel.from_pretrained("nlpaueb/legal-bert-base-uncased")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 2. Setup batching
def get_cls_embeddings(texts, batch_size=16):
    all_embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            cls_embeddings = outputs.pooler_output.detach().cpu().numpy()
            all_embeddings.append(cls_embeddings)
    return np.vstack(all_embeddings)  # Shape: (num_samples, 768)

# 3. Encode multi-label targets
mlb = MultiLabelBinarizer()
mlb.fit(train_labels_b + val_labels_b + test_labels_b)  # Fit on all splits

# 4. Prepare each split (Example: Train set)
print("Processing train set...")
train_embeddings = get_cls_embeddings(train_texts_b, batch_size=16)
train_targets = mlb.transform(train_labels_b)  # Shape: (num_samples, num_classes)

print("Processing validation set...")
val_embeddings = get_cls_embeddings(val_texts_b, batch_size=16)
val_targets = mlb.transform(val_labels_b)

print("Processing test set...")
test_embeddings = get_cls_embeddings(test_texts_b, batch_size=16)
test_targets = mlb.transform(test_labels_b)

pca = PCA(n_components=200)
train_embeddings_pca = pca.fit_transform(train_embeddings)
val_embeddings_pca = pca.transform(val_embeddings)
test_embeddings_pca = pca.transform(test_embeddings)

print("Shapes:")
print("Train embeddings:", train_embeddings_pca.shape)
print("Train targets:", train_targets.shape)
print("Val embeddings:", val_embeddings_pca.shape)
print("Val targets:", val_targets.shape)
print("Test embeddings:", test_embeddings_pca.shape)
print("Test targets:", test_targets.shape)

# Now you have (embeddings, targets) pairs for ML modeling!
# Example for torch/keras: X = train_embeddings, y = train_targets


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Processing train set...


100%|██████████| 63/63 [00:40<00:00,  1.57it/s]


Processing validation set...


100%|██████████| 13/13 [00:07<00:00,  1.84it/s]


Processing test set...


100%|██████████| 13/13 [00:06<00:00,  1.92it/s]


Shapes:
Train embeddings: (1000, 200)
Train targets: (1000, 109)
Val embeddings: (200, 200)
Val targets: (200, 109)
Test embeddings: (200, 200)
Test targets: (200, 109)


In [ ]:
def ContinousCrossover(posA, posB, gamma=0.4):
    alpha = np.random.uniform(-gamma, 1 + gamma, size=posA.shape)
    child1 = alpha * posA + (1 - alpha) * posB
    child2 = alpha * posB + (1 - alpha) * posA
    return child1, child2

def ContinousMutation(position, problem, sigma=0.1):
    VarMin = problem['VarMin']
    VarMax = problem['VarMax']
    noise = np.random.normal(scale=sigma, size=position.shape)
    mutated = position + noise
    return np.clip(mutated, VarMin, VarMax)

def printingperiter(problem, it, gbest, namemethod, funcevals, curtrial, population=None):
    print(f"[{namemethod}] Iter {it} | Evals {funcevals} | Best cost {gbest['cost']:.5f}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
import numpy as np

def rf_classifier_cost_function(particle, problem, pos=None):
    """
    Cost function for RandomForest, used by the Mayfly optimizer.
    We now optimize:
      - pos[0]: log10(n_estimators)  -> integer 10 to 1000
      - pos[1]: max_depth            -> integer 2 to 20
      - pos[2]: min_samples_split    -> integer 2 to 20
    Cost = 1 - F1_micro(validation)
    """
    if pos is None:
        pos = np.array(particle['position']).flatten()

    # Decode hyperparameters
    log10_n_estimators = float(pos[0])
    max_depth_cont     = float(pos[1])
    mss_cont           = float(pos[2])

    # n_estimators
    n_estimators = int(np.round(10 ** log10_n_estimators))
    n_estimators = int(np.clip(n_estimators, 10, 1000))

    # max_depth
    max_depth = int(np.round(np.clip(max_depth_cont, 2, 20)))

    # min_samples_split
    min_samples_split = int(np.round(np.clip(mss_cont, 2, 20)))

    try:
        base_rf = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            n_jobs=-1,
            random_state=42
        )

        clf = MultiOutputClassifier(base_rf)

        clf.fit(problem['Xtrain'], problem['Ytrain'])
        preds = clf.predict(problem['Xval'])

        f1_micro = f1_score(problem['Yval'], preds, average='micro', zero_division=0)
        cost = 1.0 - f1_micro  # lower is better

        particle['model'] = clf
        particle['hyperparams'] = {
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split
        }
        particle['cost'] = cost

        return particle
    except Exception as e:
        print('Error with RandomForest:', e)
        particle['cost'] = np.inf
        return particle


In [ ]:
import time

def MA(problem, IterPrint, MaxIter, MaxFuncEvals, curtrial, initialpop,
       mPopSize, fPopSize, a1=1.0, a2=1.5, a3=1.5, beta=2, dance=5, fl=1,
       dance_damp=0.8, fl_damp=0.99, nc=10, gmax=0.8, gmin=0.8, gamma=0.4):

    namemethod = 'MA'
    VarMin = problem['VarMin']
    VarMax = problem['VarMax']
    nVar = problem['nVar']
    VelMax = 0.1 * (VarMax - VarMin)
    VelMin = -VelMax
    g = gmax
    funcevals = -1
    start_time = time.time()

    convergence_cost, convergence_f1 = [], []
    CostFunction = problem['CostFunction']
    gbest = {'position': None, 'cost': np.inf}

    # --- initialize ---
    pop, popf = [], []
    for i in range(mPopSize):
        pos = initialpop[i]['position']
        particle = {'position': pos, 'velocity': np.zeros(nVar)}
        particle = CostFunction(particle, problem)
        particle['best_position'] = pos.copy()
        particle['best_cost'] = particle['cost']
        if particle['cost'] < gbest['cost']:
            gbest = particle.copy()
        pop.append(particle)
        funcevals += 1

    for i in range(fPopSize):
        pos = initialpop[i + mPopSize]['position']
        particle = {'position': pos, 'velocity': np.zeros(nVar)}
        particle = CostFunction(particle, problem)
        if particle['cost'] < gbest['cost']:
            gbest = particle.copy()
        popf.append(particle)
        funcevals += 1

    it = 0
    while funcevals < MaxFuncEvals - 1 and it < MaxIter:
        it += 1

        # --- update females ---
        for i in range(fPopSize):
            if popf[i]['cost'] > pop[i]['cost']:
                rmf = abs(pop[i]['position'] - popf[i]['position'])
                popf[i]['velocity'] = g*popf[i]['velocity'] + a3*np.exp(-beta * rmf**2)*(pop[i]['position'] - popf[i]['position'])
            else:
                popf[i]['velocity'] = g*popf[i]['velocity'] + fl*np.random.uniform(-1,1,size=(nVar))
            popf[i]['velocity'] = np.clip(popf[i]['velocity'], VelMin, VelMax)
            popf[i]['position'] = np.clip(popf[i]['position'] + popf[i]['velocity'], VarMin, VarMax)
            popf[i] = CostFunction(popf[i], problem)
            funcevals += 1
            if popf[i]['cost'] < gbest['cost']:
                gbest = popf[i].copy()

        # --- update males ---
        for i in range(mPopSize):
            if pop[i]['cost'] > gbest['cost']:
                rpbest = abs(pop[i]['best_position'] - pop[i]['position'])
                rgbest = abs(gbest['position'] - pop[i]['position'])
                pop[i]['velocity'] = (g*pop[i]['velocity']
                    + a1*np.exp(-beta*rpbest**2)*(pop[i]['best_position'] - pop[i]['position'])
                    + a2*np.exp(-beta*rgbest**2)*(gbest['position'] - pop[i]['position']))
            else:
                pop[i]['velocity'] = g*pop[i]['velocity'] + dance*np.random.uniform(-1,1,size=(nVar))
            pop[i]['velocity'] = np.clip(pop[i]['velocity'], VelMin, VelMax)
            pop[i]['position'] = np.clip(pop[i]['position'] + pop[i]['velocity'], VarMin, VarMax)
            pop[i] = CostFunction(pop[i], problem)
            funcevals += 1
            if pop[i]['cost'] < pop[i]['best_cost']:
                pop[i]['best_position'] = pop[i]['position'].copy()
                pop[i]['best_cost'] = pop[i]['cost']
                if pop[i]['best_cost'] < gbest['cost']:
                    gbest = pop[i].copy()

        # --- parameter update ---
        g = gmax - ((gmax - gmin) / MaxFuncEvals) * funcevals
        dance *= dance_damp
        fl *= fl_damp

        # --- Log per iteration ---
        history_iter.append(it)

        pos = np.array(gbest['position']).flatten()
        history_log10_n_estimators.append(float(pos[0]))
        history_max_depth.append(float(pos[1]))
        history_min_samples_split.append(float(pos[2]))
        history_f1micro.append(1.0 - gbest['cost'])
        history_iter.append(it)


        if it % IterPrint == 0:
            printingperiter(problem, it, gbest, namemethod, funcevals, curtrial)

    elapsed_time = time.time() - start_time
    return namemethod, gbest['position'], gbest['cost'], elapsed_time


In [ ]:
# We now optimize 3 hyperparameters:
#   pos[0]: log10(n_estimators)     -> n_estimators in [10^1, 10^3] = 10–1000
#   pos[1]: max_depth               -> [2, 20]
#   pos[2]: min_samples_split       -> [2, 20]

dim = 3

VarMin = np.array([1.0, 2.0, 2.0])    # lower bounds
VarMax = np.array([3.0, 20.0, 20.0])  # upper bounds

problem = {
    'VarMin': VarMin,
    'VarMax': VarMax,
    'nVar': dim,
    'CostFunction': rf_classifier_cost_function,
    'Xtrain': train_embeddings_pca,
    'Ytrain': train_targets,
    'Xval': val_embeddings_pca,
    'Yval': val_targets,
    'StopCriterion': 'Iterations'
}

# Initial population
mPopSize = 25
fPopSize = 25
initialpop = []
for _ in range(mPopSize + fPopSize):
    pos = VarMin + (VarMax - VarMin) * np.random.rand(dim)
    initialpop.append({'position': pos})


In [ ]:
# GLOBAL TRACKERS for RandomForest Mayfly
history_log10_n_estimators = []  # pos[0]
history_max_depth = []           # pos[1]
history_min_samples_split = []   # pos[2]
history_f1micro = []
history_iter = []


In [ ]:
# Remove labels that are constant (all 0 or all 1)
sum_per_col = problem['Ytrain'].sum(axis=0)
mask = (sum_per_col != 0) & (sum_per_col != problem['Ytrain'].shape[0])

problem['Ytrain'] = problem['Ytrain'][:, mask]
problem['Yval']   = problem['Yval'][:, mask]

print("Kept", mask.sum(), "labels (removed", (~mask).sum(), "constant ones).")

IterPrint = 1
MaxIter = 10
MaxFuncEvals = 1500
curtrial = 1

res = MA(problem, IterPrint, MaxIter, MaxFuncEvals, curtrial, initialpop,
         mPopSize=mPopSize, fPopSize=fPopSize,
         a1=1.0, a2=1.5, a3=1.5, beta=2, dance=5, fl=1,
         dance_damp=0.8, fl_damp=0.99, nc=8, gmax=0.8, gmin=0.2, gamma=0.4)

namemethod, best_position, best_cost, elapsed_time = res

# Decode 3 RF hyperparams
best_log10_n_estimators = float(best_position[0])
best_max_depth_cont     = float(best_position[1])
best_mss_cont           = float(best_position[2])

best_n_estimators    = int(np.clip(np.round(10 ** best_log10_n_estimators), 10, 1000))
best_max_depth       = int(np.clip(np.round(best_max_depth_cont), 2, 20))
best_min_samples_split = int(np.clip(np.round(best_mss_cont), 2, 20))

print("\n✅ Optimization complete (RandomForest)")
print("Best n_estimators        =", best_n_estimators)
print("Best max_depth           =", best_max_depth)
print("Best min_samples_split   =", best_min_samples_split)
print("Best cost:", best_cost, "=> Best F1_micro:", 1 - best_cost)
print("Elapsed time (s):", elapsed_time)


Kept 96 labels (removed 13 constant ones).
Single XGBoost fit took: 22.406083345413208 seconds
Single XGBoost fit took: 29.29612398147583 seconds
Single XGBoost fit took: 35.78815150260925 seconds
Single XGBoost fit took: 29.29738450050354 seconds
Single XGBoost fit took: 13.125570058822632 seconds
Single XGBoost fit took: 33.92222809791565 seconds
Single XGBoost fit took: 12.065368175506592 seconds
Single XGBoost fit took: 11.553357601165771 seconds
Single XGBoost fit took: 30.46051287651062 seconds
Single XGBoost fit took: 28.372154474258423 seconds
Single XGBoost fit took: 28.39618158340454 seconds
Single XGBoost fit took: 12.038315057754517 seconds
Single XGBoost fit took: 12.23536229133606 seconds
Single XGBoost fit took: 11.682385206222534 seconds
Single XGBoost fit took: 29.453702926635742 seconds
Single XGBoost fit took: 29.157683849334717 seconds
Single XGBoost fit took: 28.228788137435913 seconds
Single XGBoost fit took: 11.783010005950928 seconds
Single XGBoost fit took: 17.

In [ ]:
best_log10_n_estimators = float(best_position[0])
best_max_depth_cont     = float(best_position[1])
best_mss_cont           = float(best_position[2])

best_n_estimators      = int(np.clip(np.round(10 ** best_log10_n_estimators), 10, 1000))
best_max_depth         = int(np.clip(np.round(best_max_depth_cont), 2, 20))
best_min_samples_split = int(np.clip(np.round(best_mss_cont), 2, 20))

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

X_trainval = np.vstack([train_embeddings_pca, val_embeddings_pca])
Y_trainval = np.vstack([train_targets,       val_targets])

final_rf = RandomForestClassifier(
    n_estimators=best_n_estimators,
    max_depth=best_max_depth,
    min_samples_split=best_min_samples_split,
    n_jobs=-1,
    random_state=42
)

final_model = MultiOutputClassifier(final_rf)

final_model.fit(X_trainval, Y_trainval)

test_preds = final_model.predict(test_embeddings_pca)

print("\n📊 Test Metrics (RandomForest):")
print("Test F1 macro:", f1_score(test_targets, test_preds, average='macro'))
print("Test F1 micro:", f1_score(test_targets, test_preds, average='micro'))
print("Hamming Loss:", hamming_loss(test_targets, test_preds))


XGBoostError: [12:48:41] /workspace/src/objective/regression_obj.cu:119: Check failed: is_valid: base_score must be in (0,1) for the logistic loss.
Stack trace:
  [bt] (0) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x2bdf8c) [0x7b92df4bdf8c]
  [bt] (1) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x1077654) [0x7b92e0277654]
  [bt] (2) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x1077c75) [0x7b92e0277c75]
  [bt] (3) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x6d1ab9) [0x7b92df8d1ab9]
  [bt] (4) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(+0x6f4db7) [0x7b92df8f4db7]
  [bt] (5) /usr/local/lib/python3.12/dist-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x77) [0x7b92df3caa67]
  [bt] (6) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7b9518852e2e]
  [bt] (7) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x7b951884f493]
  [bt] (8) /usr/lib/python3.12/lib-dynload/_cty

In [ ]:
min_len = min(len(history_iter),
              len(history_log10_n_estimators),
              len(history_max_depth),
              len(history_min_samples_split),
              len(history_f1micro))

history_iter               = history_iter[:min_len]
history_log10_n_estimators = history_log10_n_estimators[:min_len]
history_max_depth          = history_max_depth[:min_len]
history_min_samples_split  = history_min_samples_split[:min_len]
history_f1micro            = history_f1micro[:min_len]


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16, 8))

# ----- Plot 1: log10(n_estimators) -----
plt.subplot(2, 2, 1)
plt.plot(history_iter, history_log10_n_estimators, marker='o')
plt.xlabel("Iteration")
plt.ylabel("log10(n_estimators)")
plt.title("Evolution of n_estimators")
plt.grid(True)

# ----- Plot 2: max_depth -----
plt.subplot(2, 2, 2)
plt.plot(history_iter, history_max_depth, marker='o')
plt.xlabel("Iteration")
plt.ylabel("max_depth")
plt.title("Evolution of max_depth")
plt.grid(True)

# ----- Plot 3: min_samples_split -----
plt.subplot(2, 2, 3)
plt.plot(history_iter, history_min_samples_split, marker='o')
plt.xlabel("Iteration")
plt.ylabel("min_samples_split")
plt.title("Evolution of min_samples_split")
plt.grid(True)

# ----- Plot 4: F1-micro -----
plt.subplot(2, 2, 4)
plt.plot(history_iter, history_f1micro, marker='o')
plt.xlabel("Iteration")
plt.ylabel("F1-micro")
plt.title("F1-micro Progression")
plt.grid(True)

plt.tight_layout()
plt.show()
